In [1]:
!pip install --break-system-packages faster-whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.9 MB/s eta 0:00:00


In [3]:
!pip install --break-system-packages pyannote.audio torchaudio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.2/825.2 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from faster_whisper import WhisperModel
import torch
import re
import os
import subprocess
import json
from pyannote.audio import Pipeline

/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


In [ ]:
# Configuration
INPUT_AUDIO_PATH = "call2.wav"
CLEAN_AUDIO_PATH = "cleaned_audio_for_asr_and_diarization.wav"
HUGGING_FACE_ACCESS_TOKEN = "hf_"

In [8]:
# Load faster-whisper model
# model = WhisperModel("large-v3", device="cuda" if torch.cuda.is_available() else "cpu", compute_type="float16")
model = WhisperModel("large-v3", device="cpu", compute_type="int8")

In [9]:
model

In [10]:
def get_audio_duration(audio_path):
    """Get audio duration using ffprobe"""
    try:
        cmd = ["ffprobe", "-v", "error", "-show_entries", "format=duration",
               "-of", "default=noprint_wrappers=1:nokey=1", audio_path]
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        return float(result.stdout.strip())
    except Exception as e:
        print(f"Could not get duration: {e}")
        return 0


In [11]:
def audio_preprocessing_v1(input_path, output_path):
    """Advanced audio preprocessing with better parameters"""
    print("--- Trying Advanced Audio Preprocessing ---")

    # Improved ffmpeg command - less aggressive filtering to preserve speech
    ffmpeg_command = [
        "ffmpeg", "-i", input_path,
        "-acodec", "pcm_s16le",
        "-ac", "1",  # Mono
        "-ar", "16000",  # 16kHz sample rate
        "-af", "loudnorm=I=-23:TP=-2:LRA=7,highpass=f=80,lowpass=f=8000,afftdn=nr=10",
        "-y", output_path
    ]

    try:
        result = subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
        print("Advanced preprocessing successful")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Advanced preprocessing failed: {e.returncode}")
        return False

def audio_preprocessing_v2(input_path, output_path):
    """Simplified but effective preprocessing"""
    print("--- Trying Simplified Audio Preprocessing ---")

    ffmpeg_command = [
        "ffmpeg", "-i", input_path,
        "-acodec", "pcm_s16le",
        "-ac", "1",
        "-ar", "16000",
        "-af", "loudnorm=I=-23:TP=-2,highpass=f=100",
        "-y", output_path
    ]

    try:
        result = subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
        print("Simplified preprocessing successful")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Simplified preprocessing failed: {e.returncode}")
        return False

def audio_preprocessing_v3(input_path, output_path):
    """Basic but reliable preprocessing"""
    print("--- Trying Basic Audio Preprocessing ---")

    ffmpeg_command = [
        "ffmpeg", "-i", input_path,
        "-acodec", "pcm_s16le",
        "-ac", "1",
        "-ar", "16000",
        "-af", "loudnorm",
        "-y", output_path
    ]

    try:
        result = subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
        print("Basic preprocessing successful")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Basic preprocessing failed: {e.returncode}")
        return False

def audio_preprocessing_v4(input_path, output_path):
    """Minimal processing - just format conversion"""
    print("--- Trying Minimal Audio Processing ---")

    ffmpeg_command = [
        "ffmpeg", "-i", input_path,
        "-acodec", "pcm_s16le",
        "-ac", "1",
        "-ar", "16000",
        "-y", output_path
    ]

    try:
        result = subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
        print("Minimal processing successful")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Minimal processing failed: {e.returncode}")
        return False

def smart_audio_preprocessing(input_path, output_path):
    """Try different preprocessing methods in order of preference"""
    original_duration = get_audio_duration(input_path)
    print(f"Original audio duration: {original_duration:.2f} seconds")

    methods = [
        audio_preprocessing_v1,
        audio_preprocessing_v2,
        audio_preprocessing_v3,
        audio_preprocessing_v4
    ]

    for i, method in enumerate(methods, 1):
        if method(input_path, output_path):
            if os.path.exists(output_path):
                processed_duration = get_audio_duration(output_path)
                print(f"Processed audio duration: {processed_duration:.2f} seconds")

                if abs(original_duration - processed_duration) < 1.0:
                    print(f"✅ Audio preprocessing successful with method {i}")
                    return True
                else:
                    print(f"⚠️  Duration mismatch with method {i}, trying next...")
                    continue

    print("❌ All preprocessing methods failed!")
    return False

In [12]:
def post_process_text(text):
    """Clean up transcribed text from Whisper output for call center insurance context."""
    if not text:
        return ""

    # === 1. Remove excessive immediate repetitions ===
    words = text.split()
    cleaned_words = []
    i = 0
    while i < len(words):
        current_word = words[i].lower()
        repetition_count = 1
        j = i + 1
        while j < len(words) and words[j].lower() == current_word:
            repetition_count += 1
            j += 1

        keep_count = min(repetition_count, 2) if repetition_count <= 3 else 1
        for _ in range(keep_count):
            cleaned_words.append(words[i])
        i += repetition_count

    text = ' '.join(cleaned_words)

    # === 2. Remove filler sounds (non-verbal, repetitive) ===
    filler_sounds = ["uh", "um", "mm", "hmm", "ah", "oh", "huh", "ha ha"]
    soft_fillers = ["okay okay", "yes yes", "yes yes yes", "i mean", "you know", "like like", "ok ok"]

    for filler in filler_sounds + soft_fillers:
        text = re.sub(rf'\b{re.escape(filler)}\b', '', text, flags=re.IGNORECASE)

    # === 3. Insurance domain term normalization ===
    corrections = {
        'access max life': 'Axis Max Life',
        'axis max life': 'Axis Max Life',
        'g pay': 'GPay',
        'google pay': 'Google Pay',
        'phone pay': 'PhonePe',
        'phone pe': 'PhonePe',
        'pay tm': 'Paytm',
        'net banking': 'netbanking',
        'some assured': 'sum assured',
        'premium do': 'premium due',
        'do date': 'due date',
        'okay sir': 'Okay sir',
    }

    text_lower = text.lower()
    for wrong, correct in corrections.items():
        text_lower = text_lower.replace(wrong, correct)

    # === 3.5 Replace 'Rs.', 'Rs' → '₹' with optional space cleanup ===
    text_lower = re.sub(r'\brs[.]?\s*', '₹', text_lower)

    # === 4. Punctuation cleanup ===
    text_lower = re.sub(r'\s{2,}', ' ', text_lower)          # Extra spaces
    text_lower = re.sub(r'[,]{2,}', ',', text_lower)         # Repeated commas
    text_lower = re.sub(r'\s+,', ',', text_lower)            # Space before comma
    text_lower = re.sub(r'\s+\.', '.', text_lower)           # Space before period
    text_lower = re.sub(r'\s+[!?]', lambda m: m.group(0).strip(), text_lower)

    # === 5. Capitalize sentences ===
    text_lower = re.sub(r'(^|[.!?]\s+)([a-z])',
                        lambda m: m.group(1) + m.group(2).upper(),
                        text_lower)

    return text_lower.strip()

In [13]:
def enhanced_whisper_transcription(audio_path):
    """
    Enhanced Whisper transcription with faster-whisper
    """
    print("--- Enhanced Faster-Whisper Transcription (Optimal Single Strategy) ---")

    prompt = (
        "This is a customer support call for Axis Maxlife Insurance. "
        "We will discuss policy numbers, due date, fund value, sum assured, late fee, "
        "and payment methods such as Google Pay, PhonePe, Paytm and net banking."
    )

    try:
        segments, info = model.transcribe(
            audio_path,
            language="ta",
            task="translate",
            beam_size=5,
            temperature=0.0,
            initial_prompt=prompt,
            vad_filter=True,
            vad_parameters=dict(min_silence_duration_ms=500),
            condition_on_previous_text=False,
            word_timestamps=False
        )

        whisper_result = {
            "segments": [
                {
                    "text": segment.text,
                    "start": segment.start,
                    "end": segment.end
                } for segment in segments
            ],
            "duration": info.duration,
            "language": info.language
        }

        print("✅ Faster-Whisper transcription completed with optimal parameters")
        return whisper_result

    except Exception as e:
        print(f"❌ Faster-Whisper transcription failed: {e}")
        raise

In [14]:
def calculate_repetition_score(segments):
    """
    Calculate a repetition score for transcription segments
    Lower score = less repetition = better
    """
    if not segments:
        return 0.0

    total_repetition = 0
    total_words = 0

    for segment in segments:
        text = segment.get('text', '').strip().lower()
        words = text.split()

        if len(words) < 2:
            continue

        total_words += len(words)

        # Count immediate word repetitions
        for i in range(len(words) - 1):
            if words[i] == words[i + 1]:
                total_repetition += 1

        # Count phrase repetitions within segment
        for phrase_len in range(2, min(len(words)//2 + 1, 6)):
            for start in range(len(words) - phrase_len * 2 + 1):
                phrase1 = ' '.join(words[start:start + phrase_len])
                phrase2 = ' '.join(words[start + phrase_len:start + phrase_len * 2])
                if phrase1 == phrase2:
                    total_repetition += phrase_len * 2  # Heavy penalty

    return total_repetition / max(total_words, 1)

def detect_and_remove_repetitions(segments, max_repetition_ratio=0.3):
    """
    AGGRESSIVE post-processing function to detect and remove repetitive segments
    """
    print("🔍 Starting aggressive repetition detection...")
    cleaned_segments = []

    for i, segment in enumerate(segments):
        text = segment['text'].strip()
        words = text.split()

        # Skip very short segments
        if len(words) < 2:
            continue

        # AGGRESSIVE: Check for excessive word repetition
        is_repetitive = False

        # Count word frequencies
        word_counts = {}
        for word in words:
            word_lower = word.lower().strip('.,!?')
            word_counts[word_lower] = word_counts.get(word_lower, 0) + 1

        # Check if any single word dominates the segment
        max_word_count = max(word_counts.values()) if word_counts else 0
        word_dominance = max_word_count / len(words) if words else 0

        if word_dominance > 0.4:  # If any word is >40% of the segment
            print(f"🚫 Rejecting word-dominated segment: {text[:50]}... (dominance: {word_dominance:.2f})")
            continue

        # Check for immediate repetitions (same word repeated consecutively)
        consecutive_repeats = 0
        max_consecutive = 0

        for j in range(1, len(words)):
            if words[j].lower().strip('.,!?') == words[j-1].lower().strip('.,!?'):
                consecutive_repeats += 1
                max_consecutive = max(max_consecutive, consecutive_repeats + 1)
            else:
                consecutive_repeats = 0

        if max_consecutive > 3:  # More than 3 consecutive identical words
            print(f"🚫 Rejecting consecutive repeat segment: {text[:50]}... (max consecutive: {max_consecutive})")
            continue

        # Check for pattern repetitions within segment
        for phrase_len in range(2, min(len(words)//3 + 1, 8)):
            for start in range(len(words) - phrase_len * 2 + 1):
                phrase1 = ' '.join(words[start:start + phrase_len]).lower()
                phrase2 = ' '.join(words[start + phrase_len:start + phrase_len * 2]).lower()

                if phrase1 == phrase2:
                    repetition_coverage = (phrase_len * 2) / len(words)
                    if repetition_coverage > max_repetition_ratio:
                        print(f"🚫 Rejecting pattern repeat segment: {text[:50]}... (coverage: {repetition_coverage:.2f})")
                        is_repetitive = True
                        break
            if is_repetitive:
                break

        if is_repetitive:
            continue

        # Check for similarity with recent segments (avoid near-duplicates)
        is_near_duplicate = False
        for prev_segment in cleaned_segments[-5:]:  # Check last 5 segments
            prev_words = prev_segment['text'].lower().split()
            current_words = [w.lower() for w in words]

            if prev_words and current_words:
                # Calculate Jaccard similarity
                prev_set = set(prev_words)
                current_set = set(current_words)
                intersection = len(prev_set.intersection(current_set))
                union = len(prev_set.union(current_set))

                similarity = intersection / union if union > 0 else 0

                if similarity > 0.7 and abs(len(prev_words) - len(current_words)) < 5:
                    print(f"🚫 Rejecting near-duplicate: {text[:30]}... (similarity: {similarity:.2f})")
                    is_near_duplicate = True
                    break

        if is_near_duplicate:
            continue

        # If we reach here, the segment passed all checks
        cleaned_segments.append(segment)

    removed_count = len(segments) - len(cleaned_segments)
    print(f"📊 Aggressive cleaning: {len(segments)} → {len(cleaned_segments)} segments")
    print(f"🗑️  Removed {removed_count} repetitive/problematic segments")

    return cleaned_segments

In [15]:
def main():
    """Main processing pipeline with repetition prevention"""
    print("🎯 Starting Enhanced Audio Processing Pipeline (Anti-Repetition)")
    print("=" * 60)

    # Step 1: Smart Audio Preprocessing
    if not smart_audio_preprocessing(INPUT_AUDIO_PATH, CLEAN_AUDIO_PATH):
        print("❌ Audio preprocessing failed completely. Exiting.")
        return

    # model = whisper.load_model("large")

    # Step 2: Enhanced Whisper Transcription with anti-repetition
    try:
        whisper_result = enhanced_whisper_transcription(CLEAN_AUDIO_PATH)
        print("✅ Whisper transcription completed")
    except Exception as e:
        print(f"❌ Whisper transcription failed: {e}")
        return

    # Step 3: Remove repetitive segments BEFORE post-processing
    print("\n--- Removing Repetitive Segments ---")
    cleaned_segments = detect_and_remove_repetitions(whisper_result["segments"])

    # Step 4: Post-process remaining transcription
    processed_segments = []
    for segment in cleaned_segments:
        processed_text = post_process_text(segment['text'])
        if processed_text.strip() and len(processed_text.strip()) > 5:  # Only keep meaningful segments
            segment_copy = segment.copy()
            segment_copy['text'] = processed_text
            processed_segments.append(segment_copy)

    whisper_result["segments"] = processed_segments

    # Step 5: Speaker Diarization
    print("\n--- Speaker Diarization ---")
    try:
        pipeline = Pipeline.from_pretrained(
            "pyannote/speaker-diarization-3.1",
            use_auth_token=HUGGING_FACE_ACCESS_TOKEN
        )

        if torch.cuda.is_available():
            pipeline.to(torch.device("cuda"))
            print("✅ Using GPU for diarization")

        diarization = pipeline(CLEAN_AUDIO_PATH)
        print("✅ Speaker diarization completed")

    except Exception as e:
        print(f"⚠️  Speaker diarization failed: {e}")
        diarization = None

    # Step 6: Generate Enhanced Dialogue
    print("\n--- Generating Dialogue ---")

    def get_dominant_speaker(start_time, end_time, diarization_result):
        if not diarization_result:
            return "Speaker_Unknown"

        speakers = {}
        for segment, _, speaker in diarization_result.itertracks(yield_label=True):
            overlap_start = max(start_time, segment.start)
            overlap_end = min(end_time, segment.end)
            overlap_duration = max(0, overlap_end - overlap_start)

            if overlap_duration > 0:
                speakers[speaker] = speakers.get(speaker, 0) + overlap_duration

        return max(speakers, key=speakers.get) if speakers else "Speaker_Unknown"

    # Combine segments by speaker
    dialogue = []
    current_speaker = None
    current_texts = []
    current_start = 0
    current_end = 0

    for segment in processed_segments:
        start = segment['start']
        end = segment['end']
        text = segment['text'].strip()

        speaker = get_dominant_speaker(start, end, diarization)

        # Merge consecutive segments from same speaker (within 3 seconds)
        if (speaker == current_speaker and
            current_speaker and
            (start - current_end) < 3.0):
            current_texts.append(text)
            current_end = end
        else:
            # Save previous speaker's dialogue
            if current_speaker and current_texts:
                combined_text = ' '.join(current_texts)
                # Final check for repetition in combined text
                if len(combined_text.strip()) > 10:  # Only keep substantial dialogue
                    dialogue.append({
                        'speaker': current_speaker,
                        'text': combined_text,
                        'start_time': current_start,
                        'end_time': current_end
                    })

            # Start new speaker segment
            current_speaker = speaker
            current_texts = [text]
            current_start = start
            current_end = end

    # Add final segment
    if current_speaker and current_texts:
        combined_text = ' '.join(current_texts)
        if len(combined_text.strip()) > 10:
            dialogue.append({
                'speaker': current_speaker,
                'text': combined_text,
                'start_time': current_start,
                'end_time': current_end
            })

    # Step 7: Display Results
    print("\n" + "🎭 DIALOGUE OUTPUT" + "=" * 40)

    for entry in dialogue:
        timestamp = f"[{entry['start_time']:.1f}s - {entry['end_time']:.1f}s]"
        print(f"\n{entry['speaker']} {timestamp}:")
        print(f"  📝 {entry['text']}")

    # Step 8: Save Results
    output_data = {
        'metadata': {
            'total_duration': whisper_result.get('duration', 0),
            'total_speakers': len(set(d['speaker'] for d in dialogue)),
            'total_segments': len(dialogue),
            'model_used': 'whisper-large',
            'processing_successful': True,
            'anti_repetition_applied': True
        },
        'dialogue': dialogue,
        'raw_transcription': whisper_result
    }

    with open('enhanced_transcription_results.json', 'w', encoding='utf-8') as f:
        json.dump(output_data, f, indent=2, ensure_ascii=False)

    print(f"\n💾 Results saved to: enhanced_transcription_results.json")
    print("✅ Processing completed successfully!")

if __name__ == "__main__":
    main()

🎯 Starting Enhanced Audio Processing Pipeline (Anti-Repetition)
Original audio duration: 190.76 seconds
--- Trying Advanced Audio Preprocessing ---
Advanced preprocessing successful
Processed audio duration: 190.74 seconds
✅ Audio preprocessing successful with method 1
--- Enhanced Faster-Whisper Transcription (Optimal Single Strategy) ---
✅ Faster-Whisper transcription completed with optimal parameters
✅ Whisper transcription completed

--- Removing Repetitive Segments ---
🔍 Starting aggressive repetition detection...
🚫 Rejecting near-duplicate: But don't be tensed when you s... (similarity: 0.92)
🚫 Rejecting near-duplicate: No, I don't have any.... (similarity: 1.00)
🚫 Rejecting word-dominated segment: Thank you.... (dominance: 0.50)
📊 Aggressive cleaning: 54 → 44 segments
🗑️  Removed 10 repetitive/problematic segments

--- Speaker Diarization ---


config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoi

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)
/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/io.py:85: UserWarning: torchaudio._backend.utils.info has been

✅ Speaker diarization completed

--- Generating Dialogue ---

🎭 DIALOGUE OUTPUT========================================

SPEAKER_00 [1.1s - 12.3s]:
  📝 Hello! Greetings! My name is sumathi. We have called from licensure. This is an email call. You have taken a policy from axis maxlife insurance. Can you speak for 2 minutes? Yes, madam. Quick call.

SPEAKER_01 [12.3s - 17.3s]:
  📝 I can't get the call. If i cancel the policy, will it be refunded?

SPEAKER_00 [17.3s - 33.5s]:
  📝 Okay. It will be refunded in 30 seconds. Your policy is being re-researched. I will inform you. Okay. Can you speak in tamil only? Okay. Can you tell me what reason you are not connecting?

SPEAKER_01 [33.5s - 35.5s]:
  📝 It is not possible.

SPEAKER_00 [39.7s - 56.7s]:
  📝 I hope you understand the situation. You have already paid for 1 year. Don't be tensed when you surrender after paying for the first time. Because you should have paid for at least 3 years to generate cash surrender value. Usually, if you sur